In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import time

from torch_geometric.datasets import TUDataset
import torchmetrics

import torchhd
from torchhd import embeddings
from torchhd.models import Centroid

import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

import pandas as pd
from sklearn.manifold import TSNE, MDS
import numpy as np
from ast import literal_eval


import matplotlib.pyplot as plt
import matplotlib
import numpy as np

Using cuda device


In [67]:
def degree_centrality(G):
    """
    Compute the degree centrality for nodes.
    """
    _, columns = G.edge_index
    degree = torch.bincount(columns, minlength=G.num_nodes)
    return degree / G.num_nodes


def to_undirected(edge_index):
    """
    Returns the undirected edge_index
    [[0, 1], [1, 0]] will result in [[0], [1]]
    """
    edge_index = edge_index.sort(dim=0)[0]
    edge_index = torch.unique(edge_index, dim=1)
    return edge_index


def min_max_graph_size(graph_dataset):
    if len(graph_dataset) == 0:
        return None, None

    max_num_nodes = float("-inf")
    min_num_nodes = float("inf")

    for G in graph_dataset:
        num_nodes = G.num_nodes
        max_num_nodes = max(max_num_nodes, num_nodes)
        min_num_nodes = min(min_num_nodes, num_nodes)

    return min_num_nodes, max_num_nodes


class Encoder(nn.Module):
    def __init__(self, out_features, size):
        super(Encoder, self).__init__()
        self.out_features = out_features
        self.node_ids = embeddings.Random(size, out_features)

    def forward(self, x):
        pr = degree_centrality(x)
        _, pr_argsort = pr.sort()

        node_id_hvs = torch.zeros((x.num_nodes, self.out_features), device=device)
        node_id_hvs[pr_argsort] = self.node_ids.weight[: x.num_nodes]

        row, col = to_undirected(x.edge_index)

        hvs = torchhd.bind(node_id_hvs[row], node_id_hvs[col])
        return torchhd.multiset(hvs)

In [68]:
DIMENSIONS = 10000

dataset = "MUTAG"
print(f"Testing {dataset}")

graphs = TUDataset("../data", dataset)
train_size = int(0.7 * len(graphs))
test_size = len(graphs) - train_size

train_ld, test_ld = torch.utils.data.random_split(graphs, [train_size, test_size])

min_graph_size, max_graph_size = min_max_graph_size(graphs)

encoder = Encoder(DIMENSIONS, max_graph_size)
encoder = encoder.to(device)

model = Centroid(DIMENSIONS, graphs.num_classes)
model = model.to(device)

start = time.time()

with torch.no_grad():
    for samples in tqdm(train_ld, desc="Training"):
        samples.edge_index = samples.edge_index.to(device)
        samples.y = samples.y.to(device)

        test_samples_hv = encoder(samples).unsqueeze(0)
        model.add_online(test_samples_hv, samples.y)


f1score = torchmetrics.F1Score("multiclass", num_classes=graphs.num_classes)

model.normalize()
matrix = torch.zeros((len(test_ld), DIMENSIONS))
incorrect = torch.zeros((0,), dtype=torch.int)
true_labels = torch.zeros((len(test_ld),))


for index, samples in enumerate(tqdm(test_ld, desc="Testing_")):
    samples.edge_index = samples.edge_index.to(device)

    test_samples_hv = encoder(samples).unsqueeze(0)
    matrix[index] = test_samples_hv
    test_outputs = model(test_samples_hv, dot=True)

    true_labels[index] = samples.y.item()

    f1score.update(test_outputs.cpu(), samples.y)
    if test_outputs.cpu().argmax() != samples.y:
        incorrect = torch.cat((incorrect, torch.tensor([index])), 0)
        # incorrect_prediction[index] = index


end = time.time()

print(f"Test: f1-score of {f1score.compute().item() * 100:.3f}%")
print(f"Time: {(end - start):.3f}s")

Testing MUTAG


Training:   0%|          | 0/131 [00:00<?, ?it/s]

Testing_: 100%|██████████| 57/57 [00:00<00:00, 1073.23it/s]

Test: f1-score of 68.421%
Time: 0.191s


In [80]:
from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, metric="cosine", random_state=42, perplexity=5)
vis_dims = tsne.fit_transform(matrix)
colors = ["blue", "green"]
x = np.array([x for x, _ in vis_dims])
y = np.array([y for _, y in vis_dims])
color_indices = true_labels

colormap = matplotlib.colors.ListedColormap(colors)

mask = np.ones(len(test_ld), dtype=bool)
mask[incorrect] = False

plt.scatter(x[mask], y[mask], c=color_indices[mask], cmap=colormap, alpha=0.3)

plt.scatter(x[incorrect], y[incorrect], c=color_indices[incorrect], cmap=colormap, alpha=0.6, marker="x")

plt.show(block=True)

TypeError: locally_linear_embedding() missing 1 required positional argument: 'X'

In [72]:
from IPython.display import clear_output

DIMENSIONS = 10000

dataset = "PROTEINS"
# print(f"Testing {dataset}")

graphs = TUDataset("../data", dataset)
train_size = int(0.7 * len(graphs))
test_size = len(graphs) - train_size

train_ld, test_ld = torch.utils.data.random_split(graphs, [train_size, test_size])

min_graph_size, max_graph_size = min_max_graph_size(graphs)

encoder = Encoder(DIMENSIONS, max_graph_size)
encoder = encoder.to(device)

model = Centroid(DIMENSIONS, graphs.num_classes)
model = model.to(device)

start = time.time()

with torch.no_grad():
    for samples in tqdm(train_ld, desc="Training"):
        clear_output(wait=True)
        samples.edge_index = samples.edge_index.to(device)
        samples.y = samples.y.to(device)

        test_samples_hv = encoder(samples).unsqueeze(0)
        model.add_online(test_samples_hv, samples.y)

        ######################################################################################

        f1score = torchmetrics.F1Score("multiclass", num_classes=graphs.num_classes)

        # model.normalize()
        matrix = torch.zeros((len(test_ld), DIMENSIONS))
        incorrect = torch.zeros((0,), dtype=torch.int)
        true_labels = torch.zeros((len(test_ld),))

        for index, samples in enumerate(tqdm(test_ld, desc="Testing_", disable=True)):
            samples.edge_index = samples.edge_index.to(device)

            test_samples_hv = encoder(samples).unsqueeze(0)
            matrix[index] = test_samples_hv
            test_outputs = model(test_samples_hv, dot=True)

            true_labels[index] = samples.y.item()

            f1score.update(test_outputs.cpu(), samples.y)
            if test_outputs.cpu().argmax() != samples.y:
                incorrect = torch.cat((incorrect, torch.tensor([index])), 0)
                # incorrect_prediction[index] = index

        # tsne = TSNE(n_components=2, metric="cosine", random_state=42, perplexity=5)
        tsne = MDS(n_components=2, random_state=42)
        vis_dims = tsne.fit_transform(matrix)
        colors = ["blue", "green"]
        x = np.array([x for x, _ in vis_dims])
        y = np.array([y for _, y in vis_dims])
        color_indices = true_labels

        colormap = matplotlib.colors.ListedColormap(colors)

        mask = np.ones(len(test_ld), dtype=bool)
        mask[incorrect] = False

        plt.scatter(x[mask], y[mask], c=color_indices[mask], cmap=colormap, alpha=0.3)

        plt.scatter(x[incorrect], y[incorrect], c=color_indices[incorrect], cmap=colormap, alpha=0.6, marker="x")

        plt.show()
        print(f"Test: f1-score of {f1score.compute().item() * 100:.3f}%")

/home/p4ssenger/.local/lib/python3.10/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
Training:   1%|          | 5/779 [00:10<25:52,  2.01s/it]


KeyboardInterrupt: 